### Set the run number here; decides which directory to save model and results - should match the number on the corresponding yaml file for the model run

In [21]:
run_num = 6

In [22]:
%reload_ext autoreload
import os
curr_dir = "/home/mushaikh/private/PointCloudPosePrediction"
# curr_dir = "/Users/mustafa/Documents/grad school/spring 2024/ECE285 - Visual Learning/PointCloudPosePrediction"
os.chdir(curr_dir)
import importlib
import time
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import open3d as o3d
from scipy.io import loadmat
from scipy.spatial.transform import Rotation as R
from tqdm import tqdm
from typing import Any, Callable, Optional, Tuple
import yaml
import cv2
import matplotlib.pyplot as plt
import core.pointnet as pnet
import core.deep_pose_estimator as dpe
import core.loss as dpe_loss
import utils.dataloader
import utils.utils
import utils.vision
importlib.reload(dpe)
importlib.reload(pnet)
importlib.reload(utils.utils)
importlib.reload(utils.vision)
importlib.reload(utils.dataloader)

<module 'utils.dataloader' from '/home/mushaikh/private/PointCloudPosePrediction/utils/dataloader.py'>

In [23]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
else:
    torch.manual_seed(42)

cuda


In [24]:
# load params for current model run; choose model run below after creating yaml file for it
with open("./models/run_{x}/run_{x}.yaml".format(x=run_num)) as f:
    params = yaml.safe_load(f)

In [25]:
# create data loader for Eden data - outputs pairs of point clouds and relative poses
batch_size = params['train']['batch_size']
# "/Users/mustafa/Documents/grad school/spring 2024/ECE285 - Visual Learning/eden/sample"
dataset = utils.dataloader.EdenDataset("/home/mushaikh/private/sample",
                                            keypt_method=params['keypoint_detection']['detection_method'],
                                            pcd_num_pts = params['keypoint_detection']['pcd_num_pts']
                                            )

train_inds, val_inds = torch.utils.data.random_split(dataset, [0.8,0.2])

train_sampler = torch.utils.data.SubsetRandomSampler(train_inds.indices)
valid_sampler = torch.utils.data.SubsetRandomSampler(val_inds.indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

In [26]:
# training loop

model = dpe.DeepPoseEstimator(num_pts=params['keypoint_detection']['pcd_num_pts'], 
                              use_pca = params['model']['use_pca'])
model.to(device)
lr = params['train']['lr']
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

list_train_loss = []
list_val_loss = []

for epoch in range(params['train']['epochs']):
    print("Epoch {}".format(epoch))
    running_train_loss = 0
    
    for i, data in enumerate(tqdm(train_loader)):
        
        # prepare data
        pcd1, pcd2, quat, translation = data["pcd1"], data["pcd2"], data["quat"], data["translation"]
        pcd1, pcd2 = pcd1.permute(0,2,1), pcd2.permute(0,2,1)
        target = torch.cat((translation.transpose(1,0), quat.transpose(1,0))).transpose(1,0)
        
        # move to gpu
        pcd1, pcd2, quat, translation, target = pcd1.to(device), pcd2.to(device), quat.to(device), translation.to(device), target.to(device)
        
        # reset gradients
        optimizer.zero_grad()
        
        # set mode to train - sets batchnorm/dropout policy etc.
        model.train()
        
        # get preds from model
        pred, trans_feat_1, trans_feat_2 = model(pcd1, pcd2) # predicts 7-length vector of translation/quaternion
        
        # compute loss
        loss = dpe_loss.PoseLoss(pred, target) 
#         + \
#             pnet.feature_transform_regularizer(trans_feat_1) * 0.001 + \
#                 pnet.feature_transform_regularizer(trans_feat_1) * 0.001
        
        # backward pass
        loss.backward()
        
        # gradient descent step 
        optimizer.step()
        
        running_train_loss += loss.item()
    
    avg_train_loss = running_train_loss / (i + 1)
    print('Avg. train loss: %f' % (avg_train_loss))
    list_train_loss.append(avg_train_loss)
    
    # update the learning rate scheduler
    scheduler.step()
    
    # check validation loss after every epoch
    
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization
    model.eval()
    running_vloss = 0.0
    
    # Disable gradient computation and reduce memory consumption
    with torch.no_grad():
        for i, vdata in enumerate(val_loader):
            pcd1, pcd2, quat, translation = vdata["pcd1"], vdata["pcd2"], vdata["quat"], vdata["translation"]
            pcd1, pcd2 = pcd1.permute(0,2,1), pcd2.permute(0,2,1)
            target = torch.cat((translation.transpose(1,0), quat.transpose(1,0))).transpose(1,0)
            pcd1, pcd2, quat, translation, target = pcd1.to(device), pcd2.to(device), quat.to(device), translation.to(device), target.to(device)
            pred, trans_feat_1, trans_feat_2 = model(pcd1, pcd2)
            vloss = dpe_loss.PoseLoss(pred, target) 
#             + \
#                 pnet.feature_transform_regularizer(trans_feat_1) * 0.001 + \
#                 pnet.feature_transform_regularizer(trans_feat_1) * 0.001
            running_vloss += vloss.item()
            
    avg_vloss = running_vloss / (i + 1)
    print('Avg. validation loss: {}'.format(avg_vloss))
    list_val_loss.append(avg_vloss)
    
    # save the model after every epoch
    torch.save(model.state_dict(), '{y}/run_{x}/run_{x}_deep_pose_{z}.pth'.format(
        x=run_num, y=os.path.join(curr_dir,"models"), z=epoch))

    
# plot the loss curves
fig, ax = plt.subplots(1,1)
ax.plot(range(params['train']['epochs']), list_train_loss, label="train loss")
ax.plot(range(params['train']['epochs']), list_val_loss, label="validation loss")
ax.set_xlabel("Epoch #")
ax.set_ylabel("Loss")
fig.legend()
fig.savefig(os.path.join(curr_dir,"results","run_{x}_loss.png".format(x=run_num)))

Epoch 0


  0%|          | 0/240 [00:00<?, ?it/s]

torch.Size([5, 3, 20000])
torch.Size([5, 5, 20000])
torch.Size([5, 1, 20000]) torch.Size([5, 1, 20000])
torch.Size([5, 40000, 1])


  0%|          | 1/240 [00:01<05:53,  1.48s/it]


KeyboardInterrupt: 